# Insert Data In Our Data Base

In [1]:
import sqlite3
import pandas as pd

In [2]:
def  create_table():

    # Spécifiez le chemin vers votre fichier SQL
    fichier_sql = './create_transport.sql'

    # Connexion à la base de données SQLite (le fichier sera créé s'il n'existe pas)
    connexion = sqlite3.connect('./SQL_DB/transport.db')

    # Créez un objet curseur pour exécuter des commandes SQL
    curseur = connexion.cursor()
    # Lisez le fichier SQL et exécutez chaque commande SQL
    with open(fichier_sql, 'r') as fichier:
        # commandes_sql = fichier.read().split(';')  # Divisez le fichier en commandes individuelles
        # for commande_sql in commandes_sql:
        #     try:
        #         curseur.execute(commande_sql)
        #     except sqlite3.Error as e:
        #         print("Erreur d'exécution de la commande SQL :", e)
        curseur.executescript(fichier.read())
    # Validez les changements et fermez la connexion
    connexion.commit()
    connexion.close()


In [3]:
create_table()

Now read the data base

In [4]:
# Connexion à la base de données
conn = sqlite3.connect('./SQL_DB/transport.db')

# Création d'un objet curseur
cursor = conn.cursor()

# Exécution d'une requête pour obtenir la liste des tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Récupération des résultats
tables = cursor.fetchall()

# Affichage des noms des tables
list_tables = [table[0] for table in tables]
print(list_tables)
# Fermeture de la connexion
conn.close()


['agency', 'routes', 'trips', 'stop_times', 'stops']


In [5]:
# Connexion à la base de données SQLite (le fichier sera créé s'il n'existe pas)
connexion = sqlite3.connect('./SQL_DB/transport.db')

df_list = []
for table in list_tables:
    sql_query = f'''
    SELECT * FROM {table}
    '''
    df_list.append(pd.read_sql(sql = sql_query, con = connexion))


In [6]:
print(df_list)

[Empty DataFrame
Columns: [agency_id, agency_name, agency_url, agency_timezone, agency_lang, agency_phone, agency_fare_url, agency_email]
Index: [], Empty DataFrame
Columns: [route_id, uid, agency_id, trip_id, route_short_name, route_long_name, route_type, route_url, route_color, route_text_color, route_sort_order, continuous_pickup, continuous_drop_off]
Index: [], Empty DataFrame
Columns: [trip_id, route_id, service_id, trip_headsign, trip_short_name, block_id, shape_id, direction_id, continuous_drop_off, wheelchair_accessible, bikes_allowed]
Index: [], Empty DataFrame
Columns: [trip_id, stop_times_id, stop_headsign, stop_id, departure_time, arrival_time, stop_sequence, pickup_type, drop_off_type, continuous_pickup, continuous_drop_off, timepoint, shape_dist_traveled]
Index: [], Empty DataFrame
Columns: [stop_times_id, stop_id, stop_code, stop_name, stop_desc, zone_id, stop_url, platform_code, parent_station, stop_lat, stop_lon, location_type, wheelchair_boarding, level_id, stop_timez

In [7]:
def csv_to_db(csv_file, connexion, table, dict_of_keys=None):
    df = pd.read_csv(csv_file)
    df.to_sql(table, con = connexion, index=False, if_exists='append', dtype=dict_of_keys)

In [8]:
data_base = "./SQL_DB/transport.db"
connexion = sqlite3.connect(data_base)
for table in list_tables:
    csv_file = f"./SQL_DB/SEM-GTFS/{table}.txt"
    csv_to_db(csv_file, connexion, table)
# Validez les changements et fermez la connexion
connexion.commit()
connexion.close()

## Bulk insert

In [9]:
create_table()

In [10]:
df = pd.read_csv("SQL_DB/SEM-GTFS/agency.txt")
a_dict = dict(df.iloc[0])

In [32]:
def gen_insert_query(table_name, a_dict):
    keys = tuple(a_dict.keys())
    values = tuple(a_dict.values())
    return f"INSERT INTO {table_name} {keys} VALUES {values} ;\n"

In [33]:
gen_insert_query('agency', a_dict)


"INSERT INTO agency ('agency_id', 'agency_name', 'agency_url', 'agency_timezone', 'agency_lang', 'agency_phone') VALUES ('SEM', 'Mobilités M - Tag', 'https://www.mobilites-m.fr/', 'Europe/Paris', 'FR', 438703870) ;"

In [13]:
# Connexion à la base de données
conn = sqlite3.connect('./SQL_DB/transport.db')

# Création d'un objet curseur
cursor = conn.cursor()

sql_insert = gen_insert_query('agency', a_dict)
# Exécution d'une requête pour obtenir la liste des tables
cursor.execute(sql_insert)
conn.commit()
conn.close()

In [34]:
for index, row in df_routes.iterrows():
    print(dict(row))
    break

{'agency_id': 'SEM', 'route_id': '1', 'route_short_name': 'C1', 'route_long_name': "GRENOBLE Cité Jean Macé / MEYLAN Maupertuis / MONTBONNOT-SAINT- MARTIN Pré de l'Eau", 'route_type': 3, 'route_color': 'F5D24D', 'route_text_color': '000000'}


In [42]:
def get_insert_queries(table_name, data_frame):
    return [gen_insert_query(table_name, dict(row)) for index, row in data_frame.iterrows()]
#Would give higher performances to have INSERT INTO table_name (attributs_name .. ) VALUES (attributs_values), (attributs_values) etc...

In [43]:
# DataFrame routes
df_routes = pd.read_csv("SQL_DB/SEM-GTFS/routes.txt")

list_queries = get_insert_queries("routes", df_routes)

print(list_queries[0])


INSERT INTO routes ('agency_id', 'route_id', 'route_short_name', 'route_long_name', 'route_type', 'route_color', 'route_text_color') VALUES ('SEM', '1', 'C1', "GRENOBLE Cité Jean Macé / MEYLAN Maupertuis / MONTBONNOT-SAINT- MARTIN Pré de l'Eau", 3, 'F5D24D', '000000') ;


In [44]:
def gen_insert_file(file_name, table_name,df):
    with open(file_name, 'w') as file:
        for insert in get_insert_queries(table_name, df):
            file.write(insert)

In [45]:
gen_insert_file("SQL_DB/route_insert.sql", "routes", df_routes)

## Add the primary key and foreign key afterwads

En effet, SQLite impose certaines restrictions lors de l'utilisation de ALTER TABLE pour ajouter des contraintes FOREIGN KEY. En général, la contrainte FOREIGN KEY doit faire partie de la définition de la table lors de sa création et ne peut pas être ajoutée ultérieurement à l'aide de ALTER TABLE.

## Passer par une table temporaire est une option